# Simulación de circuitos en PySpice
PySpice es una interfaz en Python para simular circuitos eléctricos utilizando el motor de simulación NgSpice. Es como un puente que te permite usar Python (lenguaje moderno y flexible) para controlar y analizar simulaciones de circuitos SPICE (Simulation Program with Integrated Circuit Emphasis). NgSpice es una versión moderna y de código abierto del simulador SPICE. Es muy potente y se usa ampliamente para simular circuitos analógicos y digitales mixtos, así como para realizar análisis de corriente, voltaje, transitorios, AC, DC, ruido, entre otros.

## Instrucciones para su instalación
Para su correcta instalación `deberá ejecutar cada una de las celdas de forma individual` en la sección de instalación, ya que si usa la función de ejecutar todo ocurriran problemas durante el proceso.
Para la ejecución de cada celda, no procederá a la siguiente hasta que la celda actual que ejecutó se vea de la siguiente forma, es decir, con una palomita de color verde. ![image.png](https://miro.medium.com/v2/resize:fit:720/format:webp/1*Wj0rKPrG_Ire5XTUh_0kLA.png)

## Recursos de apoyo
*[Documentación de la versión 1.5 de PySpice](https://pyspice.fabrice-salvaire.fr/pages/documentation.html)

*[Manual de usuario de Ngspice version 34](https://sourceforge.net/projects/ngspice/files/ng-spice-rework/old-releases/34/ngspice-34-manual.pdf/download)

*[Documentación de la API PySpice](https://pyspice.fabrice-salvaire.fr/releases/v1.5/api/PySpice.html)

> ### Que es una API
La documentación de una API es un conjunto de instrucciones, descripciones y ejemplos que explican cómo usar esa API correctamente.
* En el caso de PySpice, la documentación te dice:
* Qué funciones, clases y métodos están disponibles.
* Qué parámetros debes pasar a cada función.
* Qué tipo de datos regresa cada función.
* Qué errores pueden ocurrir y cómo manejarlos.
* Ejemplos prácticos de cómo usar la API en proyectos reales.

## Advertencia ![](https://img.icons8.com/?size=100&id=CTr8yPJXyPcs&format=png&color=000000)
Si dejas de usar un script en Google Colab (por ejemplo, cierras la pestaña o dejas de interactuar durante un tiempo), la sesión se desconectará y se perderá toda la información almacenada en la memoria temporal, incluyendo variables, archivos cargados y cualquier instalación previa (como bibliotecas o configuraciones personalizadas).
Cuando vuelvas a abrir el script o intentes continuar, tendrás que volver a ejecutar todas las celdas necesarias (Una por una) desde el principio para que el entorno quede configurado correctamente.

# Instalación

In [ ]:
!uname -a

In [ ]:
!cat /etc/os-release

In [ ]:
!sudo sed -i 's|^deb-src https://r2u.stat.illinois.edu/ubuntu jammy main|# deb-src https://r2u.stat.illinois.edu/ubuntu jammy main|' /etc/apt/sources.list.d/*.list

In [ ]:
!grep -r "r2u.stat.illinois.edu" /etc/apt/

In [ ]:
!grep -r "^deb-src.*r2u.stat.illinois.edu" /etc/apt/

In [ ]:
!sudo sed -i '/^deb-src.*r2u.stat.illinois.edu/ s/^/#/' /etc/apt/sources.list /etc/apt/sources.list.d/*.list

In [ ]:
!grep -r "r2u.stat.illinois.edu" /etc/apt/

In [ ]:
!sudo apt update

In [ ]:
!apt update && apt -q install libc6-dev

In [ ]:
!wget http://ftp.fr.debian.org/debian/pool/main/n/ngspice/libngspice0_34+ds-1_amd64.deb && apt install -q ./libngspice0_34+ds-1_amd64.deb
!wget http://ftp.fr.debian.org/debian/pool/main/n/ngspice/libngspice0-dev_34+ds-1_amd64.deb && apt install -q ./libngspice0-dev_34+ds-1_amd64.deb

In [ ]:
!dpkg -L libngspice0

In [ ]:
!pip install git+https://github.com/FabriceSalvaire/PySpice

In [ ]:
!pip list

In [ ]:
!pyspice-post-installation --check-install

In [ ]:
!git clone https://github.com/FabriceSalvaire/PySpice.git

In [ ]:
!cp -r /content/PySpice/examples/libraries /content/

In [ ]:
!pip install pint

# Ejemplo

## Declaración de librerias
Una vez declaradas las librerias y modulos, no será necesario que las vuelvan a importar

In [ ]:
import matplotlib.pyplot as plt
import pint

import PySpice.Logging.Logging as Logging
logger = Logging.setup_logging()


from PySpice.Doc.ExampleTools import find_libraries
from PySpice.Probe.Plot import plot
from PySpice.Spice.Library import SpiceLibrary
from PySpice.Spice.Netlist import Circuit
from PySpice.Unit import*

## Declaración del PATH que incluye los modelos spice
Este path incluye los modelos spice, es decir todos los parametros necesarios para simular dicho componete, los tipos de formato se muestran a continuación información que pueden encontrar en la API.

[Tipos de extenciones para modelos SPICE](https://pyspice.fabrice-salvaire.fr/releases/v1.5/_modules/PySpice/Spice/Library.html#SpiceLibrary)

EXTENSIONS =
(
* '.spice',
* '.lib',
* '.mod',
* '.lib@xyce',
* '.mod@xyce',
)

In [ ]:
spice_library = SpiceLibrary("/content/libraries")

## Ejemplo 1

Ejemplo tomado de la sección 8.8.3 de [8.Ejemplos](https://pyspice.fabrice-salvaire.fr/releases/v1.5/examples/fundamental-laws/voltage-current-divider.html)

In [ ]:
circuit = Circuit('Voltage Divider')

circuit.V('input', 1, circuit.gnd, 10@u_V)
circuit.R(1, 1, 2, 2@u_kΩ)
circuit.R(2, 2, circuit.gnd, 1@u_kΩ)

simulator = circuit.simulator(temperature=25, nominal_temperature=25)
analysis = simulator.operating_point()

for node in analysis.nodes.values():
    print('Node {}: {:5.2f} V'.format(str(node), float(node)))

circuit = Circuit('Current Divider')

circuit.I('input', 1, circuit.gnd, 1@u_A) # Fixme: current value
circuit.R(1, 1, circuit.gnd, 2@u_kΩ)
circuit.R(2, 1, circuit.gnd, 1@u_kΩ)

for resistance in (circuit.R1, circuit.R2):
    resistance.minus.add_current_probe(circuit) # to get positive value

simulator = circuit.simulator(temperature=25, nominal_temperature=25)
analysis = simulator.operating_point()

# Fixme: current over resistor
for node in analysis.branches.values():
    print('Node {}: {:5.2f} A'.format(str(node), float(node))) # Fixme: format value + unit

## Ejemplo 2

Ejemplo tomado de la sección 8.4.4 de [8.Ejemplos](https://pyspice.fabrice-salvaire.fr/releases/v1.5/examples/diode/rectification.html)

In [ ]:
figure1, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(20, 10))

circuit = Circuit('half-wave rectification')
circuit.include(spice_library['1N4148'])
source = circuit.SinusoidalVoltageSource('input', 'ent', circuit.gnd, amplitude=10, frequency=50)
circuit.X('D1', '1N4148', 'ent', 'output')
circuit.R('load', 'output', circuit.gnd, 100@u_Ω)

simulator = circuit.simulator(temperature=25, nominal_temperature=25)
analysis = simulator.transient(step_time=source.period/200, end_time=source.period*2)

ax1.set_title('Half-Wave Rectification')
ax1.set_xlabel('Time [s]')
ax1.set_ylabel('Voltage [V]')
ax1.grid()
ax1.plot(analysis['ent'])
ax1.plot(analysis.output)
ax1.legend(('input', 'output'), loc=(.05,.1))
ax1.set_ylim(float(-source.amplitude*1.1), float(source.amplitude*1.1))

circuit.C('1', 'output', circuit.gnd, 1@u_mF)

simulator = circuit.simulator(temperature=25, nominal_temperature=25)
analysis = simulator.transient(step_time=source.period/200, end_time=source.period*2)

ax2.set_title('Half-Wave Rectification with filtering')
ax2.set_xlabel('Time [s]')
ax2.set_ylabel('Voltage [V]')
ax2.grid()
ax2.plot(analysis['ent'])
ax2.plot(analysis.output)
ax2.legend(('input', 'output'), loc=(.05,.1))
ax2.set_ylim(float(-source.amplitude*1.1), float(source.amplitude*1.1))


circuit = Circuit('half-wave rectification')
circuit.include(spice_library['1N4148'])
source = circuit.SinusoidalVoltageSource('input', 'ent', circuit.gnd, amplitude=10, frequency=50)
circuit.X('D1', '1N4148', 'ent', 'output_plus')
circuit.R('load', 'output_plus', 'output_minus', 100@u_Ω)
circuit.X('D2', '1N4148', 'output_minus', circuit.gnd)
circuit.X('D3', '1N4148', circuit.gnd, 'output_plus')
circuit.X('D4', '1N4148', 'output_minus', 'ent')

simulator = circuit.simulator(temperature=25, nominal_temperature=25)
analysis = simulator.transient(step_time=source.period/200, end_time=source.period*2)

ax3.set_title('Full-Wave Rectification')
ax3.set_xlabel('Time [s]')
ax3.set_ylabel('Voltage [V]')
ax3.grid()
ax3.plot(analysis['ent'])
ax3.plot(analysis.output_plus - analysis.output_minus)
ax3.legend(('input', 'output'), loc=(.05,.1))
ax3.set_ylim(float(-source.amplitude*1.1), float(source.amplitude*1.1))

circuit.C('1', 'output_plus', 'output_minus', 1@u_mF)

simulator = circuit.simulator(temperature=25, nominal_temperature=25)
analysis = simulator.transient(step_time=source.period/200, end_time=source.period*2)

ax4.set_title('Full-Wave Rectification with filtering')
ax4.set_xlabel('Time [s]')
ax4.set_ylabel('Voltage [V]')
ax4.grid()
ax4.plot(analysis['ent'])
ax4.plot(analysis.output_plus - analysis.output_minus)
ax4.legend(('input', 'output'), loc=(.05,.1))
ax4.set_ylim(float(-source.amplitude*1.1), float(source.amplitude*1.1))

plt.tight_layout()
circuit = Circuit('115/230V Rectifier')
circuit.include(spice_library['1N4148'])
on_115 = True # switch to select 115 or 230V
if on_115:
    node_230 = circuit.gnd
    node_115 = 'node_115'
    amplitude = 115@u_V
else:
    node_230 = 'node_230'
    node_115 = circuit.gnd
    amplitude = 230@u_V
source = circuit.SinusoidalVoltageSource('input', 'ent', circuit.gnd, amplitude=amplitude, frequency=50) # Fixme: rms
circuit.X('D1', '1N4148', 'ent', 'output_plus')
circuit.X('D3', '1N4148', node_230, 'output_plus')
circuit.X('D2', '1N4148', 'output_minus', node_230)
circuit.X('D4', '1N4148', 'output_minus', 'ent')
circuit.C('1', 'output_plus', node_115, 1@u_mF)
circuit.C('2', node_115, 'output_minus', 1@u_mF)
circuit.R('load', 'output_plus', 'output_minus', 10@u_Ω)

simulator = circuit.simulator(temperature=25, nominal_temperature=25)
if on_115:
    simulator.initial_condition(node_115=0)
analysis = simulator.transient(step_time=source.period/200, end_time=source.period*2)

figure2, ax = plt.subplots(figsize=(20, 10))
ax.set_title('115/230V Rectifier')
ax.set_xlabel('Time [s]')
ax.set_ylabel('Voltage [V]')
ax.grid()
ax.plot(analysis['ent'])
ax.plot(analysis.output_plus - analysis.output_minus)
ax.legend(('input', 'output'), loc=(.05,.1))
ax.set_ylim(float(-source.amplitude*1.1), float(source.amplitude*1.1))

plt.tight_layout()